<a href="https://colab.research.google.com/github/Kim-JH-01/AI_lab_Juhun/blob/main/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# ==================================
# 1. 환경 설정 및 라이브러리 준비
# ==================================
print("--- 1. 환경 설정 시작 ---")
# 텍스트를 발음 기호로 변환하는 데 필요한 라이브러리 설치
!pip install -q deep_phonemizer

# FFmpeg 설치 (torchaudio가 오디오 포맷을 다루기 위해 필요)
!apt-get -qq update && apt-get -qq install -y ffmpeg

# 필수 라이브러리 임포트
import torch
import torchaudio
import torchaudio.transforms as T
import matplotlib.pyplot as plt
from IPython.display import Audio
from pathlib import Path

# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print("✅ 환경 설정 완료!")


# ==================================
# 2. 텍스트 입력 및 스펙트로그램 생성 (공통 과정)
# ==================================
print("\n--- 2. 텍스트 입력 및 스펙트로그램 생성 ---")

text = "Hello, my name is Juhun Kim. I am junior at Chungbuk National University, majoring in Software. I live in Cheongju, South Korea. Currently, I am working on an assignment for my Artificial Intelligence class, which involves creating a self-introduction using Text-to-Speech technology."

print("입력된 텍스트:")
print(text)

# 음소(phoneme) 기반 번들을 사용합니다.
bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_PHONE_LJSPEECH

# 텍스트 처리기 및 Tacotron2 모델 로드
processor = bundle.get_text_processor()
tacotron2 = bundle.get_tacotron2().to(device)

# 텍스트를 숫자 시퀀스로 변환하고 스펙트로그램 생성
with torch.inference_mode():
    processed, lengths = processor(text)
    processed = processed.to(device)
    lengths = lengths.to(device)
    spec, spec_lengths, _ = tacotron2.infer(processed, lengths)

print("\n✅ 스펙트로그램 생성 완료!")


# ==================================
# 3. 세 종류의 보코더로 음성 파형 생성 및 저장
# ==================================
print("\n--- 3. 보코더를 이용한 음성 생성 ---")

# --- 3-1. WaveRNN Vocoder ---
print("\n[WaveRNN 보코더로 생성 중...]")
wavernn_vocoder = bundle.get_vocoder().to(device)
sample_rate = wavernn_vocoder.sample_rate

with torch.inference_mode():
    waveform_wavernn, _ = wavernn_vocoder(spec, spec_lengths)

wavernn_path = Path("introduction_full_wavernn.wav")
torchaudio.save(wavernn_path, waveform_wavernn.cpu(), sample_rate=sample_rate)
print(f"✅ WaveRNN 결과 저장 완료: {wavernn_path} ({sample_rate} Hz)")
display(Audio(waveform_wavernn.cpu(), rate=sample_rate))


# --- 3-2. Griffin-Lim Vocoder ---
print("\n[Griffin-Lim 보코더로 생성 중...]")
gl_bundle = torchaudio.pipelines.TACOTRON2_GRIFFINLIM_PHONE_LJSPEECH
griffinlim_vocoder = gl_bundle.get_vocoder().to(device)

with torch.inference_mode():
    waveform_gl, _ = griffinlim_vocoder(spec, spec_lengths)

gl_path = Path("introduction_full_griffinlim.wav")
torchaudio.save(gl_path, waveform_gl.cpu(), sample_rate=sample_rate)
print(f"✅ Griffin-Lim 결과 저장 완료: {gl_path} ({sample_rate} Hz)")
display(Audio(waveform_gl.cpu(), rate=sample_rate))


# --- 3-3. WaveGlow Vocoder ---
print("\n[WaveGlow 보코더로 생성 중...]")
waveglow = torch.hub.load(
    'NVIDIA/DeepLearningExamples:torchhub',
    'nvidia_waveglow',
    model_math="fp32",
    pretrained=True,
    progress=False,
    trust_repo=True,
).eval().to(device)

with torch.no_grad():
    waveform_waveglow = waveglow.infer(spec)

waveglow_path = Path("introduction_full_waveglow.wav")
torchaudio.save(waveglow_path, waveform_waveglow.cpu(), sample_rate=sample_rate)
print(f"✅ WaveGlow 결과 저장 완료: {waveglow_path} ({sample_rate} Hz)")
display(Audio(waveform_waveglow.cpu(), rate=sample_rate))

print("\n--- 모든 음성 파일 생성이 완료되었습니다. ---")

--- 1. 환경 설정 시작 ---
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Using device: cuda
✅ 환경 설정 완료!

--- 2. 텍스트 입력 및 스펙트로그램 생성 ---
입력된 텍스트:
Hello, my name is Juhun Kim. I am junior at Chungbuk National University, majoring in Software. I live in Cheongju, South Korea. Currently, I am working on an assignment for my Artificial Intelligence class, which involves creating a self-introduction using Text-to-Speech technology.

✅ 스펙트로그램 생성 완료!

--- 3. 보코더를 이용한 음성 생성 ---

[WaveRNN 보코더로 생성 중...]
✅ WaveRNN 결과 저장 완료: introduction_full_wavernn.wav (22050 Hz)



[Griffin-Lim 보코더로 생성 중...]
✅ Griffin-Lim 결과 저장 완료: introduction_full_griffinlim.wav (22050 Hz)



[WaveGlow 보코더로 생성 중...]


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


✅ WaveGlow 결과 저장 완료: introduction_full_waveglow.wav (22050 Hz)



--- 모든 음성 파일 생성이 완료되었습니다. ---
